In [ ]:
%pip install -U langchain jq

In [ ]:
from langchain.document_loaders import JSONLoader
from pprint import pprint

loader = JSONLoader(
    file_path='./monster_text.json', jq_schema='keys[] as $k | "\($k), \(.[$k] | .ip)')
data = loader.load()

